In [1]:
import pandas as pd
import numpy as np


In [24]:
def calculate_daily_return(row):
    return row['Close'] - row['Open']

In [115]:
def expected_value():
    pass

In [49]:
def load_stock(filename):
    data = pd.read_csv(filename)
    data['Gross Return'] = data.apply(calculate_daily_return, axis=1)
    data['Pct Return'] = data.apply(lambda row: 100*row['Gross Return']/row['Open'], axis=1)
    return data

In [59]:
nasdaq = load_stock('NASDAQ_10y.csv')
nasdaq.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Gross Return,Pct Return
0,2010-09-07,2227.260010,2231.280029,2206.620117,2208.889893,2208.889893,1685240000,-18.370117,-0.824785
1,2010-09-08,2216.070068,2237.419922,2215.899902,2228.870117,2228.870117,2018210000,12.800049,0.577601
2,2010-09-09,2251.090088,2251.979980,2229.800049,2236.199951,2236.199951,1706650000,-14.890137,-0.661463
3,2010-09-10,2239.080078,2246.600098,2229.120117,2242.479980,2242.479980,1692470000,3.399902,0.151844
4,2010-09-13,2263.800049,2289.489990,2263.689941,2285.709961,2285.709961,1937630000,21.909912,0.967838


In [86]:
def get_composite_var(composite):
    return composite.var()

def calc_cov(comp_var, stock, metric):
    var_series = pd.Series([comp_var for n in range(len(stock.index))])
    return stock[metric].cov(var_series)

def calc_B_im(stock, comp, metric, comp_var=None):
    if comp_var[metric] == None:
        comp_var = get_composite_var(comp)
    cov = calc_cov(comp_var[metric], stock, metric)
    return cov/comp_var[metric]

In [87]:
costco = load_stock('COST_10y.csv')
costco.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Gross Return,Pct Return
0,2010-09-07,58.250000,59.000000,58.200001,58.639999,45.681999,2598500,0.389999,0.669526
1,2010-09-08,59.369999,59.419998,58.799999,59.209999,46.126049,3412700,-0.160000,-0.269496
2,2010-09-09,59.500000,59.509998,58.830002,58.939999,45.915714,2622900,-0.560001,-0.941178
3,2010-09-10,59.299999,59.790001,59.139999,59.540001,46.383121,2608600,0.240002,0.404725
4,2010-09-13,59.820000,60.230000,59.410000,60.049999,46.780426,3107800,0.229999,0.384485


In [88]:
calc_B_im(costco, nasdaq, 'Pct Return')

TypeError: 'NoneType' object is not subscriptable

In [101]:
stockList = ['COST', 'INTC', 'AAPL', 'TSLA'] # must download data first
compName = 'NASDAQ'

In [112]:
comp = load_stock('{}_10y.csv'.format(compName))
comp_var = get_composite_var(comp)

stocks = pd.DataFrame({'B_im': [0 for n in range(len(stockList))],
                       'cov': [0 for n in range(len(stockList))]}, index=stockList)

for stock in stockList:
    stock_data = load_stock('{}_10y.csv'.format(stock))
    stocks.loc[stock, 'B_im'] = calc_B_im(stock_data, comp, 'Pct Return', comp_var=comp_var)
    stocks.loc[stock, 'cov'] = calc_cov(comp_var['Pct Return'], stock_data, 'Pct Return')

stocks.head()

,B_im,cov
COST,9.458335e-34,7.835329e-34
INTC,4.764636e-33,3.947047e-33
AAPL,-6.810001e-33,-5.641437e-33
TSLA,-1.059333e-32,-8.775568e-33


In [114]:
R_om = min([abs(c) for c in stocks['cov']])